In [2]:
import pandas as pd
import numpy as np

# -----------------------------
# 1. Cargar dataset
# -----------------------------
df = pd.read_csv("trigo_sintetico_mejorado.csv")

# -----------------------------
# 2. Ingeniería de características
# -----------------------------

# --- Grados Día (GDD) ---
# Base de temperatura depende de la temporada
df["GDD"] = df.apply(
    lambda row: max(0, row["Temp_media"] - (5 if row["Temporada"]=="invierno" else 10)),
    axis=1
)

# --- Rango térmico ---
df["Rango_termico"] = df["Temp_max"] - df["Temp_min"]

# --- Índice de fertilidad ---
# Optimales de N, P, K (kg/ha)
N_opt, P_opt, K_opt = 200, 90, 90
df["Fertilidad_total"] = (
    0.6 * (df["N_kg_ha"] / N_opt) +
    0.25 * (df["P_kg_ha"] / P_opt) +
    0.15 * (df["K_kg_ha"] / K_opt)
)

# --- Categoría de clima ---
def categorizar_clima(row):
    if row["Temporada"] == "invierno":
        if row["Lluvia_mm"] > 600 and row["Temp_media"] < 16:
            return "húmedo-frío"
        else:
            return "templado"
    else:  # primavera
        if row["Lluvia_mm"] < 300 and row["Temp_media"] > 18:
            return "seco-cálido"
        else:
            return "templado"
df["Categoria_clima"] = df.apply(categorizar_clima, axis=1)

# --- pH categorizado ---
df["pH_categorizado"] = pd.cut(
    df["pH"],
    bins=[0, 6.0, 7.5, 14],
    labels=["ácido", "óptimo", "alcalino"]
)

# -----------------------------
# 3. Guardar dataset enriquecido
# -----------------------------
df.to_csv("trigo_sintetico_features.csv", index=False)
print("✅ Dataset enriquecido generado: trigo_sintetico_features.csv")

# Mostrar primeras filas para verificar
print(df.head())


✅ Dataset enriquecido generado: trigo_sintetico_features.csv
   Temporada   Lluvia_mm  Horas_luz  Temp_media   Temp_max   Temp_min  \
0  primavera  274.616140  66.574556   20.930727  26.696399  15.097642   
1   invierno  748.369082  85.457170   14.591214  18.858878  10.042027   
2  primavera  231.193282  66.953086   20.463094  26.455113  16.245704   
3   invierno  641.389455  91.705628   14.855981  20.519908  10.329401   
4   invierno  615.499807  85.036914   16.404401  19.410735  11.448496   

      N_kg_ha    P_kg_ha    K_kg_ha        pH       Labranza Plagas  \
0  192.768906  75.859290  95.025058  6.503462   convencional     no   
1  162.311206  81.602488  94.379237  8.115774  cero labranza     no   
2  192.440209  92.541601  89.935371  6.069078   convencional     no   
3  217.577819  97.996140  91.569438  6.737266  cero labranza     no   
4  253.698373  96.671785  91.279580  7.301333  cero labranza     no   

  Deficiencia  Rendimiento_kg_ha        GDD  Rango_termico  Fertilidad_to